### 迭代器

迭代器比可迭代对象占用空间要少

一类是集合数据类型（list、tuple、dict、set、str）,一类是generator包括生成器和yield，这些可以直接作用于for循环的对象都统称为可迭代对象。

In [3]:
#可以使用isinstance()判断是否可迭代
from collections import Iterable
isinstance("abc",Iterable)  #True,可迭代
isinstance(100,Iterable)    #False,不可迭代

False

可迭代的不一定是可迭代对象，生成器是可迭代对象，也可以用iter()函数生成可迭代对象

In [5]:
#iter()函数创建迭代器
a = [11,22,33,44]
print(type(a))
iter(a)

<class 'list'>


### 闭包

在函数内部再定义一个函数，并且这个函数用到了外边函数的变量，那么我们将这个函数以及用到的一些变量，里面的整体为闭包

In [11]:
#闭包特点:当执行test(100)返回number为100，再次执行ret()会使用上一次的number值最后得到结果101
def test(number):
    print("---1---")
    def test_in(number2):
        print("---2---")
        print(number+number2)
    print("---3---")
    return test_in   #test_in指向test_in()函数

ret = test(100) #ret指向test_in，也就是全部指向test_in()的函数体
print("-"*10)
ret(1)


---1---
---3---
----------
---2---
101


### 闭包-应用

简化调用方式，比传统写法调用简便

In [ ]:
def test(a,b):
    def test_in(x):
        print(a*x+b)
    return test_in

line1 = test(1,1)
line1(2)


### 装饰器

在不改变原函数功能的情况下添加新功能

In [1]:
#在原功能的基础上新增验证权限

#新功能-闭包
def w1(func):
    def inner():
        print("---正在验证权限----")
        func()
    return inner
#原功能
def f1():
    print("---f1---")
#原功能
def f2():
    print("---f2---")

#innerFunc = w1(f1)
#innerFunc()

f1 = w1(f1)   #@符号等价于这句话
f1()   #使得函数f1()功能改变

---正在验证权限----
---f1---


In [3]:
def w1(func):
    def inner():
        print("---正在验证权限----")
        func()      
    return inner
@w1
def f1():     #当调用时，func()=f1()
    print("---f1---")
@w1
def f2():
    print("---f2---")

f1()
f2()

---正在验证权限----
---f1---
---正在验证权限----
---f2---


### 2个装饰器

In [9]:
#定义函数：完成包裹数据
def makeBold(fn):
    def wrapped():
        print("----1---")
        return "<b>" + fn() + "</b>"
    return wrapped

#定义函数：完成包裹数据
def makeItalic(fn):
    def wrapped():
        print("----2---")
        return "<i>" + fn() + "</i>"
    return wrapped

#先执行装饰器@makeItalic
@makeBold
@makeItalic
def test3():
    print("----3---")
    return "hello world-3"

ret = test3()
print(ret)

#执行流程:
#1.@makeBold先执行，打印 print("----1---")，return wrapped函数，fn()传入调用@makeItalic
#2.@makeItalic继续执行，打印 print("----2---")，return wrapped函数，fn()传入调用test3()，输出 print("----3---")和<i>hello world-3</i>
#3.最后@makeBold才调用@makeItalic的return值 


----1---
----2---
----3---
<b><i>hello world-3</i></b>


### 装饰器执行的时间

In [10]:
def w1(func):
    print("---正在装饰1----")
    def inner():
        print("---正在验证权限1----")
        func()
    return inner

def w2(func):
    print("---正在装饰2----")
    def inner():
        print("---正在验证权限2----")
        func()
    return inner

#只要python解释器执行到了这个代码,那么就会自动的进行装饰,而不是等到调用的时候才装饰的
@w1
@w2
def f1():
    print("---f1---")

#在调用f1之前,已经进行装饰了
f1()

---正在装饰2----
---正在装饰1----
---正在验证权限1----
---正在验证权限2----
---f1---


### 装饰器对有参数函数进行装饰

In [ ]:
#定长参数
def func(functionName):
    print("---func---1---")
    def func_in(a, b):#如果a,b 没有定义,那么会导致16行的调用失败
        print("---func_in---1---")
        functionName(a, b)#如果没有把a,b当做实参进行传递,那么会导致调用12行的函数失败
        print("---func_in---2---")

    print("---func---2---")
    return func_in

@func
def test(a, b):
    print("----test-a=%d,b=%d---"%(a,b))

test(11,22)

In [1]:
#不定长参数
def func(functionName):
    print("---func---1---")
    def func_in(*args, **kwargs):#采用不定长参数的方式满足所有函数需要参数以及不需要参数的情况
        print("---func_in---1---")
        functionName(*args, **kwargs)#这个地方,需要写*以及**,如果不写的话,那么args是元祖,而kwargs是字典
        print("---func_in---2---")

    print("---func---2---")
    return func_in

@func
def test(a, b, c):
    print("----test-a=%d,b=%d,c=%d---"%(a,b,c))

@func
def test2(a, b, c, d):
    print("----test-a=%d,b=%d,c=%d,d=%d---"%(a,b,c,d))

test(11,22,33)

test2(44,55,66,77)

---func---1---
---func---2---
---func---1---
---func---2---
---func_in---1---
----test-a=11,b=22,c=33---
---func_in---2---
---func_in---1---
----test-a=44,b=55,c=66,d=77---
---func_in---2---


### 装饰器对带有返回值的函数进行装饰

In [2]:
#返回值为空
def func(functionName):
    print("---func---1---")
    
    
    def func_in():
        print("---func_in---1---")
        functionName() 
        print("---func_in---2---")

    print("---func---2---")
    return func_in

@func
def test():
    print("----test----")
    return "haha"    #此处有返回值，但调用func_in不会被保存

ret = test()
print("test return value is %s"%ret)

---func---1---
---func---2---
---func_in---1---
----test----
---func_in---2---
test return value is None


In [3]:
#有返回值
def func(functionName):
    print("---func---1---")
    def func_in():
        print("---func_in---1---")
        ret = functionName() #保存 返回来的haha
        print("---func_in---2---")
        return ret #把haha返回岛17行处的调用

    print("---func---2---")
    return func_in

@func


def test():
    print("----test----")
    return "haha"

ret = test()
print("test return value is %s"%ret)

---func---1---
---func---2---
---func_in---1---
----test----
---func_in---2---
test return value is haha


### 通用装饰器

当有无返回值、有无参数都可以调用

In [ ]:
def func(functionName):
    def func_in(*args, **kwargs):
        print("-----记录日志-----")
        ret = functionName(*args, **kwargs)
        return ret
    return func_in

@func
def test():
    print("----test----")
    return "haha"

@func
def test2():
    print("----test2---")

@func
def test3(a):
    print("-----test3--a=%d--"%a)

ret = test()
print("test return value is %s"%ret)

a = test2()
print("test2 return value is %s"%a)

test3(11)

### 带有参数的装饰器

In [4]:
def func_arg(arg):
    def func(functionName):
        def func_in():
            print("---记录日志-arg=%s--"%arg)    #传入不同的参数
            if arg=="heihei":
                #调两次
                functionName()
                functionName()
            else:
                #调一次
                functionName()
        return func_in
    return func

#1. 先执行func_arg("heihei")函数,这个函数return 的结果是func这个函数的引用
#2. @func
#3. 使用@func对test进行装饰
@func_arg("heihei")
def test():
    print("--test--")

#带有参数的装饰器,能够起到在运行时,有不同的功能
@func_arg("haha")
def test2():
    print("--test2--")

test()
test2()

---记录日志-arg=heihei--
--test--
--test--
---记录日志-arg=haha--
--test2--


### 作用域

globals()查看全局变量和locals()查看局部变量

LEGB规则：Python使用LEGB的顺序来查找一个符号对应的对象

In [ ]:
locals:局部变量
enclosing:外部嵌套函数的命名空间
  def fun1():
    a = 10
    def fun2():
        #a处于外部嵌套函数的命名空间
glabals:全局变量
builtins:

### Python动态添加属性以及方法

#### 给对象添加属性

In [3]:
class Person(object):
    def __init__(self, newName, newAge):
        self.name = newName
        self.age = newAge

laowang = Person("老王", 10000)
print(laowang.name)
print(laowang.age)
laowang.addr = "北京...."  #添加属性
print(laowang.addr)

laozhao = Person("老赵", 18)
#print(laozhao.addr)

Person.num = 100           #添加属性
print(laowang.num)
print(laozhao.num)

老王
10000
北京....
100
100


#### 给对象添加方法

In [2]:
import types

class Person(object):
    def __init__(self, newName, newAge):
        self.name = newName
        self.age = newAge
    
    def eat(self):
        print("-----%s正在吃----"%self.name)


def run(self):
    print("-----%s正在跑----"%self.name)


p1 = Person("p1", 10)
p1.eat()
#添加方法,types.MethodType(函数名,对象名)
p1.run = types.MethodType(run,p1)
p1.run()

-----p1正在吃----
-----p1正在跑----


#### 给类添加静态属性

In [6]:
@staticmethod
def test():
    print("-----static method-----")
    
P.test = test
P.test()

-----static method-----


#### 给类添加类方法

In [7]:
@classmethod
def printnum(cls):
    print("-----class method------")
    
P.printnum = printnum
P.printnum()

-----class method------


### _ _slots_ _的作用

为了达到限制的目的，Python允许在定义class的时候，定义一个特殊的_ _slots_ _变量，来限制该class实例能添加的属性

In [11]:
class Person(object):
    __slots__ = ("name","age")

p = Person()
p.name = "老王"
p.age = 100
p.sex = "女"  #不允许添加该属性， __slots__做了限制

AttributeError: 'Person' object has no attribute 'sex'

### 生成器

通过列表生成式，可以直接创建一个列表。在Python中当列表生成式计算量太大，一边循环一边计算的机制，称为生成器：generator

#### 创建生成器方法1--把列表生成式的中括号改成小括号

In [18]:
b = (x*2 for x in range(10))
print(b)

#逐个生成,每执行一次生成一个，到最后一个还去生成会产生异常
next(b)
#next(b)
#next(b)
#next(b)

<generator object <genexpr> at 0x0000000004D50B48>


0

####  创建生成器方法2--当函数中有yield则此函数为生成器

In [20]:
#斐波那契数列，1 1 2 3 5 8 13 21 34
def creatnum():
    a,b = 0,1
    for i in range(5):
        print("---1---")
        yield b       #当运行到yield,后面不运行，并保存b并返回
        print("---2---")
        a,b = b,a+b

a = creatnum()
next(a)  #再次执行，会返回2 ，到最后一个还去生成会产生异常

---1---


1

### 生成器2--send

In [ ]:
def creatnum():
    a,b = 0,1
    for i in range(5):
        print("---1---")
        yield b       #当运行到yield,后面不运行，并保存b并返回
        print("---2---")
        a,b = b,a+b

a = creatnum()
for num in a:
    print(num)
#用for循环执行完不会产生异常

In [1]:
def creatnum():
    a,b = 0,1
    for i in range(5):
        print("---1---")
        yield b       #当运行到yield,后面不运行，并保存b并返回
        print("---2---")
        a,b = b,a+b

a = creatnum()
ret = a.__next__()   #这个方式和next(a)等价
print(ret)

---1---
1


In [4]:
def test():
    i = 1
    while i < 5:
        temp = yield i  #temp的值不是i值，而是num空值
        print(temp)
        i +=1
t = test()
t.__next__()  #再次执行会返回下一个值

1

In [ ]:
def test():
    i = 1
    while i < 5:
        temp = yield i  #temp的值不是i值，而是num空值
        print(temp)
        i +=1
t = test()
t.send("hh")  #当执行这一句会执行temp = yield i，给temp传值hh

### 生成器4--多任务

In [1]:
def test1():
    while True:
        print("---1---")
        yield None
        
def test2():
    while True:
        print("---2---")
        yield None
        
t1 = test1()
t2 = test2()
while True:
    t1.__next__()
    t2.__next__()
#三个while一直循环执行，看上去同时执行的东西就是多任务（会造成同时执行的假象，其实不是）

---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---


---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---


---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---


---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---


---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---


...

---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---


---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---


---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---


---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---


KeyboardInterrupt: 